In [1]:
import pandas as pd

In [4]:
df_rvas = pd.read_csv('/Users/liaoruqi/Desktop/structure-informed-rvas/sample_df_rvas.tsv', sep='\t')

In [5]:
df_rvas['pdb_filename'] = 'AF-O15047-F1-model_v4.pdb.gz'

In [16]:
annot_file = pd.read_csv('/Users/liaoruqi/Desktop/structure-informed-rvas/ClinVar_PLP_uniprot_canonical.tsv', sep='\t')
annot_file

,uniprot_id,aa_pos,aa_ref,aa_alt
0,Q6TDP4,561,A,E
1,P43489,65,R,C
2,Q96L58,65,S,G
3,Q96L58,67,P,L
4,Q96L58,79,T,A
...,...,...,...,...
58488,P00451,19,S,R
58489,Q96DA2,192,G,R
58490,Q96DA2,168,T,K
58491,Q96DA2,63,W,C


In [6]:
df_rvas.to_csv('/Users/liaoruqi/Desktop/structure-informed-rvas/sample_df_rvas.tsv', sep='\t', index=False)

In [2]:
import pandas as pd

df = pd.DataFrame([['AF-O15047-F1-model_v4.pdb.gz', [1, 1707]], ['AF-O15018-F1-model_v4.pdb.gz', [1, 1401]], ['AF-O15018-F2-model_v4.pdb.gz', [201, 1601]], ['AF-O15018-F3-model_v4.pdb.gz', [401, 1801]], ['AF-O15018-F4-model_v4.pdb.gz', [601, 2001]], ['AF-O15018-F5-model_v4.pdb.gz', [801, 2201]], ['AF-O15018-F6-model_v4.pdb.gz', [1001, 2401]], ['AF-O15018-F7-model_v4.pdb.gz', [1201, 2601]], ['AF-O15018-F8-model_v4.pdb.gz', [1401, 2801]], ['AF-O15018-F9-model_v4.pdb.gz', [1601, 3001]] ], columns=['filename', 'pos_covered'])
df

,filename,pos_covered
0,AF-O15047-F1-model_v4.pdb.gz,"[1, 1707]"
1,AF-O15018-F1-model_v4.pdb.gz,"[1, 1401]"
2,AF-O15018-F2-model_v4.pdb.gz,"[201, 1601]"
3,AF-O15018-F3-model_v4.pdb.gz,"[401, 1801]"
4,AF-O15018-F4-model_v4.pdb.gz,"[601, 2001]"
5,AF-O15018-F5-model_v4.pdb.gz,"[801, 2201]"
6,AF-O15018-F6-model_v4.pdb.gz,"[1001, 2401]"
7,AF-O15018-F7-model_v4.pdb.gz,"[1201, 2601]"
8,AF-O15018-F8-model_v4.pdb.gz,"[1401, 2801]"
9,AF-O15018-F9-model_v4.pdb.gz,"[1601, 3001]"


In [3]:
df.to_csv('/Users/liaoruqi/Desktop/structure-informed-rvas/info.tsv', sep='\t', index=False)

In [61]:
import os
import gzip
import shutil
from Bio.PDB import PDBParser
import ast

from Bio.PDB import StructureBuilder, PDBIO, Model, Chain

def write_full_pdb(full_pdb, output_path):
    builder = StructureBuilder.StructureBuilder()
    builder.init_structure("new_structure")
    builder.init_model(0)
    builder.init_chain('A')  

    model = builder.get_structure()[0]
    chain = model['A']

    for residue in full_pdb:
        chain.add(residue.copy())  

    io = PDBIO()
    io.set_structure(builder.get_structure())
    io.save(output_path)


def get_one_pdb(info_tsv, uniprot_id, reference_directory):
    info_df = pd.read_csv(info_tsv, sep='\t')
    print(info_df)
    info_df['pos_covered'] = info_df['pos_covered'].apply(ast.literal_eval)
    pdbs = [item for item in os.listdir(reference_directory) if item.endswith('.gz') and uniprot_id in item]
    full_pdb = []
    pdbs.sort(key=lambda item: item.split('-')[2][-1])
    print(pdbs)
    for item in pdbs:
        # print(item[:-3])
        p = os.path.join(reference_directory, item)
        # print(p_pos_cover)
        if 'F1' in item:
            with gzip.open(p, "rt") as handle:
                structure = PDBParser(QUIET=True).get_structure("protein", handle)
            for model in structure:
                for chain in model:
                    for residue in chain:
                        full_pdb.append(residue)
        else:
            with gzip.open(p, "rt") as handle:
                structure = PDBParser(QUIET=True).get_structure("protein", handle)
            new_residue = []
            for model in structure:
                for chain in model:
                    for residue in chain:
                        new_residue.append(residue)
            # print(full_pdb[-1].id)
            new_residue = new_residue[1200:]
            # print(len(new_residue))
            for i in range(len(new_residue)):
                new_residue[i].id = (' ', 1 + i + full_pdb[-1].id[1], ' ')
            full_pdb.extend(new_residue)

    write_full_pdb(full_pdb, os.path.join(reference_directory, uniprot_id + '.pdb'))

get_one_pdb('/Users/liaoruqi/Desktop/structure-informed-rvas/info.tsv', 'O15018', '/Users/liaoruqi/Desktop/structure-informed-rvas/')

                    filename   pos_covered
0           AF-O15047-F1.pdb     [1, 1707]
1  AF-O15018-F1-model_v4.pdb     [1, 1401]
2  AF-O15018-F2-model_v4.pdb   [201, 1601]
3  AF-O15018-F3-model_v4.pdb   [401, 1801]
4  AF-O15018-F4-model_v4.pdb   [601, 2001]
5  AF-O15018-F5-model_v4.pdb   [801, 2201]
6  AF-O15018-F6-model_v4.pdb  [1001, 2401]
7  AF-O15018-F7-model_v4.pdb  [1201, 2601]
8  AF-O15018-F8-model_v4.pdb  [1401, 2801]
9  AF-O15018-F9-model_v4.pdb  [1601, 3001]
['AF-O15018-F1-model_v4.pdb.gz', 'AF-O15018-F2-model_v4.pdb.gz', 'AF-O15018-F3-model_v4.pdb.gz', 'AF-O15018-F4-model_v4.pdb.gz', 'AF-O15018-F5-model_v4.pdb.gz', 'AF-O15018-F6-model_v4.pdb.gz', 'AF-O15018-F7-model_v4.pdb.gz', 'AF-O15018-F8-model_v4.pdb.gz', 'AF-O15018-F9-model_v4.pdb.gz']
(' ', 1400, ' ')
200
(' ', 1600, ' ')
200
(' ', 1800, ' ')
200
(' ', 2000, ' ')
200
(' ', 2200, ' ')
200
(' ', 2400, ' ')
200
(' ', 2600, ' ')
200
(' ', 2800, ' ')
39


In [66]:
from Bio.PDB import PDBParser, Polypeptide

aa_dict = {
    'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C',
    'GLN': 'Q', 'GLU': 'E', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I',
    'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F', 'PRO': 'P',
    'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V'
}

def get_sequence_from_pdb(pdb_file):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file)

    sequences = []
    for model in structure:
        for chain in model:
            for residue in chain:
                sequences.append(aa_dict[residue.get_resname()])
    
    return sequences  # List of sequences (could be more than one per chain if disjoint)

# Example usage
seqs = ''.join(get_sequence_from_pdb("O15018.pdb"))
seqs == 'MPITQDNAVLHLPLLYQWLQNSLQEGGDGPEQRLCQAAIQKLQEYIQLNFAVDESTVPPDHSPPEMEICTVYLTKELGDTETVGLSFGNIPVFGDYGEKRRGGKKRKTHQGPVLDVGCIWVTELRKNSPAGKSGKVRLRDEILSLNGQLMVGVDVSGASYLAEQCWNGGFIYLIMLRRFKHKAHSTYNGNSSNSSEPGETPTLELGDRTAKKGKRTRKFGVISRPPANKAPEESKGSAGCEVSSDPSTELENGPDPELGNGHVFQLENGPDSLKEVAGPHLERSEVDRGTEHRIPKTDAPLTTSNDKRRFSKGGKTDFQSSDCLAREEVGRIWKMELLKESDGLGIQVSGGRGSKRSPHAIVVTQVKEGGAAHRDGRLSLGDELLVINGHLLVGLSHEEAVAILRSATGMVQLVVASKENSAEDLLRLTSKSLPDLTSSVEDVSSWTDNEDQEADGEEDEGTSSSVQRAMPGTDEPQDVCGAEESKGNLESPKQGSNKIKLKSRLSGGVHRLESVEEYNELMVRNGDPRIRMLEVSRDGRKHSLPQLLDSSSASQEYHIVKKSTRSLSTTQVESPWRLIRPSVISIIGLYKEKGKGLGFSIAGGRDCIRGQMGIFVKTIFPNGSAAEDGRLKEGDEILDVNGIPIKGLTFQEAIHTFKQIRSGLFVLTVRTKLVSPSLTPCSTPTHMSRSASPNFNTSGGASAGGSDEGSSSSLGRKTPGPKDRIVMEVTLNKEPRVGLGIGACCLALENSPPGIYIHSLAPGSVAKMESNLSRGDQILEVNSVNVRHAALSKVHAILSKCPPGPVRLVIGRHPNPKVSEQEMDEVIARSTYQESKEANSSPGLGTPLKSPSLAKKDSLISESELSQYFAHDVPGPLSDFMVAGSEDEDHPGSGCSTSEEGSLPPSTSTHKEPGKPRANSLVTLGSHRASGLFHKQVTVARQASLPGSPQALRNPLLRQRKVGCYDANDASDEEEFDREGDCISLPGALPGPIRPLSEDDPRRVSISSSKGMDVHNQEERPRKTLVSKAISAPLLGSSVDLEESIPEGMVDAASYAANLTDSAEAPKGSPGSWWKKELSGSSSAPKLEYTVRTDTQSPTNTGSPSSPQQKSEGLGSRHRPVARVSPHCKRSEAEAKPSGSQTVNLTGRANDPCDLDSRVQATSVKVTVAGFQPGGAVEKESLGKLTTGDACVSTSCELASALSHLDASHLTENLPKAASELGQQPMTELDSSSDLISSPGKKGAAHPDPSKTSVDTGQVSRPENPSQPASPRVTKCKARSPVRLPHEGSPSPGEKAAAPPDYSKTRSASETSTPHNTRRVAALRGAGPGAEGMTPAGAVLPGDPLTSQEQRQGAPGNHSKALEMTGIHAPESSQEPSLLEGADSVSSRAPQASLSMLPSTDNTKEACGHVSGHCCPGGSRESPVTDIDSFIKELDASAARSPSSQTGDSGSQEGSAQGHPPAGAGGGSSCRAEPVPGGQTSSPRRAWAAGAPAYPQWASQPSVLDSINPDKHFTVNKNFLSNYSRNFSSFHEDSTSLSGLGDSTEPSLSSMYGDAEDSSSDPESLTEAPRASARDGWSPPRSRVSLHKEDPSESEEEQIEICSTRGCPNPPSSPAHLPTQAAICPASAKVLSLKYSTPRESVASPREKAACLPGSYTSGPDSSQPSSLLEMSSQEHETHADISTSQNHRPSCAEETTEVTSASSAMENSPLSKVARHFHSPPIILSSPNMVNGLEHDLLDDETLNQYETSINAAASLSSFSVDVPKNGESVLENLHISESQDLDDLLQKPKMIARRPIMAWFKEINKHNQGTHLRSKTEKEQPLMPARSPDSKIQMVSSSQKKGVTVPHSPPQPKTNLENKDLSKKSPAEMLLTNGQKAKCGPKLKRLSLKGKAKVNSEAPAANAVKAGGTDHRKPLISPQTSHKTLSKAVSQRLHVADHEDPDRNTTAAPRSPQCVLESKPPLATSGPLKPSVSDTSIRTFVSPLTSPKPVPEQGMWSRFHMAVLSEPDRGCPTTPKSPKCRAEGRAPRADSGPVSPAASRNGMSVAGNRQSEPRLASHVAADTAQPRPTGEKGGNIMASDRLERTNQLKIVEISAEAVSETVCGNKPAESDRRGGCLAQGNCQEKSEIRLYRQVAESSTSHPSSLPSHASQAEQEMSRSFSMAKLASSSSSLQTAIRKAEYSQGKSSLMSDSRGVPRNSIPGGPSGEDHLYFTPRPATRTYSMPAQFSSHFGREGHPPHSLGRSRDSQVPVTSSVVPEAKASRGGLPSLANGQGIYSVKPLLDTSRNLPATDEGDIISVQETSCLVTDKIKVTRRHYCYEQNWPHESTSFFSVKQRIKSFENLANADRPVAKSGASPFLSVSSKPPIGRRSSGSIVSGSLGHPGDAAARLLRRSLSSCSENQSEAGTLLPQMAKSPSIMTLTISRQNPPETSSKGSDSELKKSLGPLGIPTPTMTLASPVKRNKSSVRHTQPSPVSRSKLQELRALSMPDLDKLCSEDYSAGPSAVLFKTELEITPRRSPGPPAGGVSCPEKGGNRACPGGSGPKTSAAETPSSASDTGEAAQDLPFRRSWSVNLDQLLVSAGDQQRLQSVLSSVGSKSTILTLIQEAKAQSENEEDVCFIVLNRKEGSGLGFSVAGGTDVEPKSITVHRVFSQGAASQEGTMNRGDFLLSVNGASLAGLAHGNVLKVLHQAQLHKDALVVIKKGMDQPRPSARQEPPTANGKGLLSRKTIPLEPGIGRSVAVHDALCVEVLKTSAGLGLSLDGGKSSVTGDGPLVIKRVYKGGAAEQAGIIEAGDEILAINGKPLVGLMHFDAWNIMKSVPEGPVQLLIRKHRNSS'


True

In [69]:
pd.read_csv('/Users/liaoruqi/Desktop/structure-informed-rvas/O15047_results.tsv', sep='\t')

,aa_pos,nbhd_case,nbhd_ctrl,p_value,ratio,p_values_0,p_values_1,p_values_2
0,1,0,0,1.000000,0.25,1.000000,1.0,1.000000
1,2,0,0,1.000000,0.25,1.000000,1.0,1.000000
2,3,0,0,1.000000,0.25,1.000000,1.0,1.000000
3,4,0,0,1.000000,0.25,1.000000,1.0,1.000000
4,5,0,0,1.000000,0.25,1.000000,1.0,1.000000
...,...,...,...,...,...,...,...,...
1238,1239,2,0,0.387414,2.75,0.387414,1.0,0.570771
1239,1240,2,0,0.387414,2.75,0.387414,1.0,0.570771
1240,1241,2,0,0.387414,2.75,0.387414,1.0,0.570771
1241,1242,2,0,0.387414,2.75,0.387414,1.0,0.570771


In [85]:
import numpy as np
df = pd.DataFrame(columns=['aa_pos', 'ratio', 'p_value', 'nbhd_case', 'nbhd_ctrl'])
df['aa_pos'] = np.arange(1, 1708)
ratio = np.random.uniform(0, 12, 1707)
df['ratio'] = ratio
p_values = np.random.uniform(0, 1, 1707)
df['p_value'] = p_values
nbhd_case = np.random.randint(0, 10, 1707)
df['nbhd_case'] = nbhd_case
nbhd_ctrl = np.random.randint(0, 10, 1707)
df['nbhd_ctrl'] = nbhd_ctrl


In [86]:
df.to_csv('/Users/liaoruqi/Desktop/structure-informed-rvas/O15047_results.tsv', sep='\t', index=False)

In [ ]:
from pymol import cmd

def zoom_to_selection():
    # Get the current selection
    selection_name = "sele"  # Default name for selected atoms/residues
    if cmd.count_atoms(selection_name) > 0:
        cmd.zoom(selection_name, buffer=5)
        print(f"Zoomed to selection: {selection_name}")
    else:
        print("No selection found. Please select a residue by clicking it in PyMOL.")

# Bind this function to a custom command so it can be run in PyMOL
cmd.extend("zoomsel", zoom_to_selection)


In [2]:
from pymol import cmd

# Load the structure (replace with your actual structure file)
cmd.load("O15047.pse", 'structure')
cmd.show_as('ribbon', 'structure')
cmd.rebuild()


In [47]:
5001 - 3801 

1200

In [ ]:
    # info_df = pd.read_csv(info_tsv, sep='\t')
    # info_df['uniprot_id'] = info_df['filename'].apply(lambda x: x.split('-')[1])
    # info_df['pos_covered'] = info_df['pos_covered'].apply(ast.literal_eval)
    # info_df['start_pos'] = info_df['pos_covered'].apply(lambda x: x[0])
    # info_df['end_pos'] = info_df['pos_covered'].apply(lambda x: x[1])
    # annot_df = pd.read_csv(annot_file, sep='\t')
    # annot_uniprot = set(annot_df['uniprot_id'].tolist())
    # for item in annot_uniprot:
    #     tmp_annot = annot_df[annot_df['uniprot_id'] == item]
    #     tmp_info = info_df[info_df['uniprot_id'] == item]
    #     for _, row in tmp_annot.iterrows(): 
    #         mask = (tmp_info['start_pos'].astype(int) < int(row['aa_pos'])) & (tmp_info['end_pos'].astype(int) > int(row['aa_pos']))
    #         annot_info = tmp_info[mask]
    #         uniprot_id = row['uniprot_id']
    #         pse_filenames = [f'{uniprot_id}_{item.split('.')[0]}.pse' for item in annot_info['filename'].tolist()]
    #         for filename in pse_filenames:
    #             p = os.path.join(reference_directory, filename)
    #             cmd.load(p)
    #             cmd.select(f"annotation_residue_{row['aa_pos']}", f"resi {row['aa_pos']}")
    #             cmd.label(f"annotation_residue_{row['aa_pos']} and name CA", f'"annotation"')
    #             cmd.save(p)

In [ ]:

pymol_rvas('info.tsv','sample_df_rvas.tsv', '/Users/liaoruqi/Desktop/structure-informed-rvas/')
pymol_annotation('ClinVar_PLP_uniprot_canonical.tsv', '/Users/liaoruqi/Desktop/structure-informed-rvas/')
pymol_scan_test('O15047_results.tsv', '/Users/liaoruqi/Desktop/structure-informed-rvas/')
pymol_neighborhood('O15047_results.tsv', '/Users/liaoruqi/Desktop/structure-informed-rvas/')